In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Code for generating bitrates of an equirectangular video

In [3]:
import os
import subprocess


"""Change the input name of the file here"""
input_video = "/content/drive/MyDrive/360_Video_IS/360_london_4k_10s.mp4"

# Function to get video duration
def get_video_duration(file_path):
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-select_streams", "v:0", "-show_entries", "stream=duration", "-of", "csv=p=0", file_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )
    return float(result.stdout.strip())

def get_video_segements(file_path,segement_dir,segement_time=0.5):
    # Ensure segments directory exists
    os.makedirs(output_segments_dir, exist_ok=True)
    command = [
    "ffmpeg",
    "-i", f"{file_path}",
    "-c:v", "libx264",
    "-c:a", "aac",
    "-strict", "experimental",
    "-b:a", "192k",
    "-force_key_frames", f"expr:gte(t,n_forced*{segement_time})",
    "-f", "segment",
    "-segment_time", f"{segement_time}",
    "-reset_timestamps", "1",
    "-map", "0",
    f"{segement_dir}/output_part_%d.mp4"
    ]
    subprocess.run(command,check=True)

#Function to convert an equirectangular to cubic
def convert_video_to_cubemap(input_file, input_format="equirect", output_format="c3x2", quality=2):
    """
    Converts a video from equirectangular projection to cubemap projection using FFmpeg.

    Args:
        input_file (str): Path to the input video file.
        input_format (str): Input projection format (default is 'equirect').
        output_format (str): Output projection format (default is 'c3x2').
        quality (int): Video quality parameter (lower is better quality, default is 2).

    Returns:
        str: The FFmpeg command executed.
    """
    # Derive output file name
    base_name, ext = os.path.splitext(input_file)
    output_file = f"{base_name}_cube{ext}"

    command = [
        "ffmpeg",
        "-i", input_file,
        "-vf", f"v360=input={input_format}:output={output_format}",
        "-q:v", str(quality),
        output_file
    ]

    # Execute the FFmpeg command
    subprocess.run(command, check=True)

    return " ".join(command)

# Function to calculate bitrate
def calculate_bitrate(file_path):
    command = [
    "ffprobe",
    "-v", "error",
    "-select_streams", "v",
    "-show_entries", "stream=bit_rate",
    "-of", "default=noprint_wrappers=1",
    f"{file_path}"
    ]
    result= subprocess.run(command,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True)
    return int(result.stdout.strip().split('=')[1])


# Function to get video duration
def get_video_resolution(file_path):
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-select_streams", "v:0", "-show_entries", "stream=width,height", "-of", "csv=s=x:p=0", file_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )
    result= result.stdout.strip().split('x')
    return (int(result[0])//3,int(result[1])//2)

# Extract tiles and calculate bitrates

width,height=get_video_resolution(input_video)
convert_video_to_cubemap(input_video)
input_video=input_video.replace('.mp4','_cube.mp4')
"""Change the tile width by setting it here"""
tile_width=width//3
output_segments_dir = f"{input_video.replace('.mp4','_')}segments"
tile_dict={
    "tile_1_left.mp4":f"{tile_width}:{height}:0:0",
    "tile_2_left.mp4":f"{tile_width}:{height}:{width}:0",
    "tile_3_left.mp4":f"{tile_width}:{height}:{2*width}:0",
    "tile_4_left.mp4":f"{tile_width}:{height}:0:{height}",
    "tile_5_left.mp4":f"{tile_width}:{height}:{width}:{height}",
    "tile_6_left.mp4":f"{tile_width}:{height}:{2*width}:{height}",
    "tile_1_right.mp4":f"{tile_width}:{height}:{width-tile_width}:0",
    "tile_2_right.mp4":f"{tile_width}:{height}:{2*width-tile_width}:0",
    "tile_3_right.mp4":f"{tile_width}:{height}:{3*width-tile_width}:0",
    "tile_4_right.mp4":f"{tile_width}:{height}:{width-tile_width}:{height}",
    "tile_5_right.mp4":f"{tile_width}:{height}:{2*width-tile_width}:{height}",
    "tile_6_right.mp4":f"{tile_width}:{height}:{3*width-tile_width}:{height}"
}

# Split the tile video into 0.5 sec segments
get_video_segements(input_video,output_segments_dir)

bitrates=[]
diff_bitrates=[[0,0,0,0,0,0,0,0,0,0,0,0],]
"""
  Structure
  [segement_0 values, segment_1 values, ...]
  Each segement_no values=list of tile value differences
  Calculate bitrate for each segment
"""
segments = [f"output_part_{i}.mp4" for i in range(0,len(os.listdir(output_segments_dir)))]
for index,segment in enumerate(segments):
    bitrate_dict={}
    diff_bitrate_segment=[]
    for tile_name,crop_value in tile_dict.items():
        """
        Extract the tile_width x (height_resolution/2) size tiles
        subprocess.run(["ffmpeg", "-i", input_video, "-vf", "crop=16:16:1024:1536", output_tile])
        """
        subprocess.run(["ffmpeg", "-i", output_segments_dir+'/'+segment, "-vf", f"crop={crop_value}", output_segments_dir+'/'+segment.replace('.mp4','_')+tile_name])
        bitrate=calculate_bitrate(output_segments_dir+'/'+segment.replace('.mp4','_')+tile_name)
        bitrate_dict[tile_name]=bitrate
        if index > 0:
             diff_bitrate_segment.append(bitrate - bitrates[index-1][tile_name])
    bitrates.append(bitrate_dict)
    if index >0 :
        diff_bitrates.append(diff_bitrate_segment)
for bitrate in bitrates:
    print(bitrate)
for diff_bitrate in diff_bitrates:
    print(diff_bitrate)



{'tile_1_left.mp4': 693120, 'tile_2_left.mp4': 527520, 'tile_3_left.mp4': 545865, 'tile_4_left.mp4': 524865, 'tile_5_left.mp4': 553365, 'tile_6_left.mp4': 540645, 'tile_1_right.mp4': 535320, 'tile_2_right.mp4': 445740, 'tile_3_right.mp4': 545865, 'tile_4_right.mp4': 441990, 'tile_5_right.mp4': 530700, 'tile_6_right.mp4': 540645}
{'tile_1_left.mp4': 665632, 'tile_2_left.mp4': 533808, 'tile_3_left.mp4': 561792, 'tile_4_left.mp4': 506288, 'tile_5_left.mp4': 560960, 'tile_6_left.mp4': 553920, 'tile_1_right.mp4': 549248, 'tile_2_right.mp4': 456736, 'tile_3_right.mp4': 561792, 'tile_4_right.mp4': 449536, 'tile_5_right.mp4': 542944, 'tile_6_right.mp4': 553920}
{'tile_1_left.mp4': 1072848, 'tile_2_left.mp4': 809488, 'tile_3_left.mp4': 763472, 'tile_4_left.mp4': 806064, 'tile_5_left.mp4': 922544, 'tile_6_left.mp4': 829840, 'tile_1_right.mp4': 823552, 'tile_2_right.mp4': 654640, 'tile_3_right.mp4': 763472, 'tile_4_right.mp4': 809200, 'tile_5_right.mp4': 765776, 'tile_6_right.mp4': 829840}
{'tile

Tile index mapping

In [4]:
index_corr={
    0:'tile_1_left.mp4',
    1:'tile_2_left.mp4',
    2:'tile_3_left.mp4',
    3:'tile_4_left.mp4',
    4:'tile_5_left.mp4',
    5:'tile_6_left.mp4',
    6:'tile_1_right.mp4',
    7:'tile_2_right.mp4',
    8:'tile_3_right.mp4',
    9:'tile_4_right.mp4',
    10:'tile_5_right.mp4',
    11:'tile_6_right.mp4'
}
normalize_pattern={
    0:6,
    1:7,
    2:8,
    3:9,
    4:10,
    5:11
}

Code for calculating the most corelated tiles in the video

In [44]:
import numpy as np
import pandas as pd
most_similar_differences = []
most_similar_values=[]
threshold=0.15
df_diff_bitrates=pd.DataFrame(diff_bitrates)
for i in range(2,len(diff_bitrates)):
    curr_diff=np.array(diff_bitrates[i])
    last_diff=np.array(diff_bitrates[i-1])
    # present_dict=pd.DataFrame({'curr_diff':[curr_diff[0]],'last_diff':[last_diff[0]]})
    # print(present_dict['curr_diff'].corr(present_dict['last_diff']))
    for index,value in enumerate(curr_diff):
        """
        Correlation when all values are positive
        differences=np.abs(value * value/np.sqrt(last_diff* last_diff+value*value))
        """
        differences=last_diff/value
        """Add ignored indexes here"""
        mask=[2,4,8,10]
        if(index not in mask):
          mask.append(index)
        differences[mask]=-100
        below_threshold_indices = np.where((differences > 0) & (abs(differences-1) <= threshold) )[0]
        if len(below_threshold_indices) > 1:
            last_check_segment=i-4 if i>4 else 0
            df_corr=df_diff_bitrates[last_check_segment:i+1].corr()
            # print(i,index,below_threshold_indices)
            # print(df_corr.iloc[index,below_threshold_indices].to_numpy())
            below_threshold_indices=[below_threshold_indices[np.argmax(df_corr.iloc[index,below_threshold_indices].to_numpy())]]
        if len(below_threshold_indices) > 0:
            most_similar_values.append((i,index,below_threshold_indices,differences[below_threshold_indices]))

print("Differences of the most similar values:")
for most_similar_value in most_similar_values:
  print(f"Segment {most_similar_value[0]}, Curr: {index_corr[most_similar_value[1]]}, Prev Tile:{[index_corr[i] for i in most_similar_value[2]]}")

Differences of the most similar values:
Segment 5, Curr: tile_2_left.mp4, Prev Tile:['tile_1_left.mp4']
Segment 5, Curr: tile_5_left.mp4, Prev Tile:['tile_4_right.mp4']
Segment 5, Curr: tile_2_right.mp4, Prev Tile:['tile_6_left.mp4']
Segment 5, Curr: tile_5_right.mp4, Prev Tile:['tile_6_left.mp4']
Segment 6, Curr: tile_1_right.mp4, Prev Tile:['tile_2_right.mp4']
Segment 7, Curr: tile_1_left.mp4, Prev Tile:['tile_2_right.mp4']
Segment 7, Curr: tile_4_left.mp4, Prev Tile:['tile_1_left.mp4']
Segment 7, Curr: tile_2_right.mp4, Prev Tile:['tile_4_left.mp4']
Segment 8, Curr: tile_6_left.mp4, Prev Tile:['tile_6_right.mp4']
Segment 8, Curr: tile_6_right.mp4, Prev Tile:['tile_6_left.mp4']
Segment 10, Curr: tile_1_right.mp4, Prev Tile:['tile_1_left.mp4']
Segment 11, Curr: tile_2_left.mp4, Prev Tile:['tile_6_left.mp4']
Segment 11, Curr: tile_1_right.mp4, Prev Tile:['tile_1_left.mp4']
Segment 12, Curr: tile_2_right.mp4, Prev Tile:['tile_4_right.mp4']
Segment 12, Curr: tile_4_right.mp4, Prev Tile:[

In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
norm_diff=scaler.fit_transform(diff_bitrates)
df_norm_diff_bitrates=pd.DataFrame(diff_bitrates)

most_similar_differences = []
most_similar_values=[]
threshold=0.15
for i in range(2,len(diff_bitrates)):
    curr_diff=np.array(diff_bitrates[i])
    last_diff=np.array(diff_bitrates[i-1])
    for index,value in enumerate(curr_diff):
        differences=last_diff/value
        # differences[index]=1e10
        mask=[2,4,8,10]
        if(index not in mask):
          mask.append(index)
        differences[mask]=-100
        # min_index=np.argmin(differences)
        below_threshold_indices = np.where((differences > 0) & (abs(differences-1) <= threshold) )[0]
        if len(below_threshold_indices) > 1:
            last_check_segment=i-4 if i>4 else 0
            df_corr=df_norm_diff_bitrates[last_check_segment:i+1].corr()
            # print(i,index,below_threshold_indices)
            # print(df_corr.iloc[index,below_threshold_indices].to_numpy())
            below_threshold_indices=[below_threshold_indices[np.argmax(df_corr.iloc[index,below_threshold_indices].to_numpy())]]
        if len(below_threshold_indices) > 0:
            most_similar_values.append((i,index,below_threshold_indices,differences[below_threshold_indices]))
        # most_similar_differences.append(differences[min_index])

print("Differences of the most similar values:")
for most_similar_value in most_similar_values:
  print(f"Segment {most_similar_value[0]}, Curr: {index_corr[most_similar_value[1]]}, Prev Tile:{[index_corr[i] for i in most_similar_value[2]]}")

Differences of the most similar values:
Segment 5, Curr: tile_2_left.mp4, Prev Tile:['tile_1_left.mp4']
Segment 5, Curr: tile_5_left.mp4, Prev Tile:['tile_4_right.mp4']
Segment 5, Curr: tile_2_right.mp4, Prev Tile:['tile_6_left.mp4']
Segment 5, Curr: tile_5_right.mp4, Prev Tile:['tile_6_left.mp4']
Segment 6, Curr: tile_1_right.mp4, Prev Tile:['tile_2_right.mp4']
Segment 7, Curr: tile_1_left.mp4, Prev Tile:['tile_2_right.mp4']
Segment 7, Curr: tile_4_left.mp4, Prev Tile:['tile_1_left.mp4']
Segment 7, Curr: tile_2_right.mp4, Prev Tile:['tile_4_left.mp4']
Segment 8, Curr: tile_6_left.mp4, Prev Tile:['tile_6_right.mp4']
Segment 8, Curr: tile_6_right.mp4, Prev Tile:['tile_6_left.mp4']
Segment 10, Curr: tile_1_right.mp4, Prev Tile:['tile_1_left.mp4']
Segment 11, Curr: tile_2_left.mp4, Prev Tile:['tile_6_left.mp4']
Segment 11, Curr: tile_1_right.mp4, Prev Tile:['tile_1_left.mp4']
Segment 12, Curr: tile_2_right.mp4, Prev Tile:['tile_4_right.mp4']
Segment 12, Curr: tile_4_right.mp4, Prev Tile:[

In [9]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 16.7 MB/s eta 0:00:00


In [10]:
import cv2
from ultralytics import YOLO

# Load the model
yolo = YOLO('yolov8s.pt')

# Load the video capture (Replace 'video.mp4' with the path to your video file)
videoCap = cv2.VideoCapture(input_video)

# Get video properties
frame_width = int(videoCap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(videoCap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(videoCap.get(cv2.CAP_PROP_FPS))

# Define codec and create VideoWriter object (output file: 'output.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 format
output = cv2.VideoWriter(input_video.replace('.mp4','_yolo.mp4'), fourcc, fps, (frame_width, frame_height))

# Function to get class colors
def getColours(cls_num):
    base_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    color_index = cls_num % len(base_colors)
    increments = [(1, -2, 1), (-2, 1, -1), (1, -1, 2)]
    color = [base_colors[color_index][i] + increments[color_index][i] *
    (cls_num // len(base_colors)) % 256 for i in range(3)]
    return tuple(color)

while True:
    ret, frame = videoCap.read()
    if not ret:
        break  # Exit loop if the video ends

    results = yolo.track(frame, stream=True)

    for result in results:
        # get the classes names
        classes_names = result.names

        # iterate over each box
        for box in result.boxes:
            # check if confidence is greater than 40 percent
            if box.conf[0] > 0.4:
                # get coordinates
                [x1, y1, x2, y2] = box.xyxy[0]
                # convert to int
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # get the class
                cls = int(box.cls[0])

                # get the class name
                class_name = classes_names[cls]

                # get the respective colour
                colour = getColours(cls)

                # draw the rectangle
                cv2.rectangle(frame, (x1, y1), (x2, y2), colour, 2)

                # put the class name and confidence on the image
                cv2.putText(frame, f'{classes_names[int(box.cls[0])]} {box.conf[0]:.2f}', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, colour, 2)

    # Write the processed frame to the output video file
    output.write(frame)

# Release resources
videoCap.release()
output.release()
cv2.destroyAllWindows()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 121MB/s] 


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.3 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.6s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 448x640 11 persons, 1 dog, 415.7ms
Speed: 15.4ms preprocess, 415.7ms inference, 23.9ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 11 persons, 1 dog, 293.1ms
Speed: 2.6ms preprocess, 293.1ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 11 persons, 1 dog, 291.8ms
Speed: 2.0ms preprocess, 291.8ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 11 persons, 1 dog, 300.2ms
Speed: 2.9ms preprocess, 300.2ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 11 persons, 1 dog, 309.3ms
Speed: 2.9ms preprocess, 309.3ms inference, 2.2ms postprocess per image at shape (1, 3,

In [37]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from scipy.signal import coherence
#normalizing bitrate values per tile
scaler=MinMaxScaler()
i=0
temp=[[2,4,5,12,-2,2,4,6,-6,-13,1,12],[1,4,5,12,2,2,4,6,-6,-13,1,15],[0,2,4,6,8,10,1,3,5,7,9,11]]
for diff_bitrate in temp:
    tile_set=[]
    for left_tile,right_tile in normalize_pattern.items():
        tile_set.append([diff_bitrate[left_tile],diff_bitrate[right_tile]])
        # tile_set=np.array([[diff_bitrate[left_tile]],[diff_bitrate[tile_right]]])
        # normalized_tiles_values=scaler.fit_transform(tile_set)
        # print(tile_set,normalized_tiles_values)
    normalized_tiles_values=scaler.fit_transform(np.array(tile_set))
    print(normalized_tiles_values)
    print(tile_set)
    i+=1
    if i>2:
      break

[[    0.28571        0.68]
 [    0.42857        0.76]
 [        0.5        0.28]
 [          1           0]
 [          0        0.56]
 [    0.28571           1]]
[[2, 4], [4, 6], [5, -6], [12, -13], [-2, 1], [2, 12]]
[[          0     0.60714]
 [    0.27273     0.67857]
 [    0.36364        0.25]
 [          1           0]
 [   0.090909         0.5]
 [   0.090909           1]]
[[1, 4], [4, 6], [5, -6], [12, -13], [2, 1], [2, 15]]
[[          0           0]
 [        0.2         0.2]
 [        0.4         0.4]
 [        0.6         0.6]
 [        0.8         0.8]
 [          1           1]]
[[0, 1], [2, 3], [4, 5], [6, 7], [8, 9], [10, 11]]
